In [ ]:
import pandas as pd
from google.colab import drive
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Flatten, Reshape, Dropout, BatchNormalization, MaxPooling1D
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
drive.mount('/content/drive')

df = pd.read_excel('/content/drive/MyDrive/TCC/RMS INDEPENDENTES.xlsx', sheet_name="JEITO 7")

Mounted at /content/drive


In [ ]:
X = df.iloc[:, 1:8]#.values.reshape(-1, 1)
y = df.iloc[:, 0].values

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
# Criar o modelo
modelo = Sequential([
    Conv1D(filters=8, kernel_size=8, input_shape=(X.shape[1], 1), activation='gelu', padding='same'),
    #MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(units=1, activation='relu')
])

# Compilar o modelo
modelo.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
modelo.fit(X, y, epochs=50, batch_size=32, verbose=1)

# Fazer previsões
y_pred = modelo.predict(X)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'Mean Squared Error (MSE): {mse:.3f}')
print(f'R^2 Score: {r2:.3f}')

Epoch 1/50
1519/1519 [==============================] - 3s 1ms/step - loss: 0.1820
Epoch 2/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0978
Epoch 3/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0850
Epoch 4/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0726
Epoch 5/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0653
Epoch 6/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0635
Epoch 7/50
1519/1519 [==============================] - 2s 2ms/step - loss: 0.0630
Epoch 8/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0628
Epoch 9/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0626
Epoch 10/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0622
Epoch 11/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0622
Epoch 12/50
1519/1519 [==============================] - 2s 1ms/step - loss: 0.0619
E

In [ ]:
# Função para criar o modelo
def create_model(activation_conv='tanh', pool_size=2, activation_dense='relu'):
    model = Sequential([
        Conv1D(filters=8, kernel_size=8, input_shape=(X.shape[1], 1), activation=activation_conv, padding='same'),
        MaxPooling1D(pool_size=pool_size),
        Flatten(),
        Dense(units=1, activation=activation_dense)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Definir os parâmetros para a busca em grade
param_grid = {'activation_conv': ['sigmoid', 'relu', 'tanh', 'softmax', 'swish', 'gelu'],
              'pool_size': [1, 2, 3, 4],
              'activation_dense': ['sigmoid', 'relu', 'tanh', 'softmax', 'swish', 'gelu']}

# Inicializar listas para armazenar os resultados
best_params = None
best_mse = np.inf
best_r2 = -np.inf

cont = 0

# Realizar a busca em grade manualmente
for activation_conv in param_grid['activation_conv']:
    for pool_size in param_grid['pool_size']:
        for activation_dense in param_grid['activation_dense']:
            model = create_model(activation_conv=activation_conv, pool_size=pool_size, activation_dense=activation_dense)
            model.fit(X, y, epochs=50, batch_size=32, verbose=1)
            y_pred = model.predict(X)
            mse = mean_squared_error(y, y_pred)
            r2 = r2_score(y, y_pred)
            cont += 1
            print(f"--------{cont}------------")

            # Atualizar melhores parâmetros se necessário
            if mse < best_mse:
                best_mse = mse
                best_r2 = r2
                best_params = {'activation_conv': activation_conv, 'pool_size': pool_size, 'activation_dense': activation_dense}

# Imprimir os resultados
print("Melhores Parâmetros: ", best_params)
print("Melhor MSE: ", best_mse)
print("Melhor R^2 Score: ", best_r2)

# Fazer previsões no conjunto de teste usando os melhores parâmetros encontrados
best_model = create_model(**best_params)
best_model.fit(X, y, epochs=50, batch_size=32, verbose=0)
y_pred_test = best_model.predict(X)

print(f'Mean Squared Error (MSE) no conjunto de teste: {best_mse:.3f}')
print("Melhores Parâmetros: ", best_params)
print(f'R^2 Score no conjunto de teste: {best_r2:.3f}')

Streaming output truncated to the last 5000 lines.
Epoch 26/50
1519/1519 [==============================] - 6s 4ms/step - loss: 0.3213
Epoch 27/50
1519/1519 [==============================] - 5s 3ms/step - loss: 0.3213
Epoch 28/50
1519/1519 [==============================] - 5s 4ms/step - loss: 0.3212
Epoch 29/50
1519/1519 [==============================] - 6s 4ms/step - loss: 0.3212
Epoch 30/50
1519/1519 [==============================] - 5s 4ms/step - loss: 0.3211
Epoch 31/50
1519/1519 [==============================] - 5s 3ms/step - loss: 0.3211
Epoch 32/50
1519/1519 [==============================] - 5s 3ms/step - loss: 0.3210
Epoch 33/50
1519/1519 [==============================] - 6s 4ms/step - loss: 0.3210
Epoch 34/50
1519/1519 [==============================] - 5s 3ms/step - loss: 0.3209
Epoch 35/50
1519/1519 [==============================] - 5s 3ms/step - loss: 0.3209
Epoch 36/50
1519/1519 [==============================] - 5s 3ms/step - loss: 0.3209
Epoch 37/50
1519/1519 [==

In [ ]:
print(X)